# RETRIEVING DATA FROM TWITTER

In [2]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For number computing

# For plotting and visualization:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import csv
import mysql.connector

from nltk import bigrams
import networkx as nx

from sqlalchemy import create_engine
import pymysql

In [ ]:
# Consume:
CONSUMER_KEY    = 'iaW7mNbmXJNzK7H0lYp9w9lvl'
CONSUMER_SECRET = '39od1VNgE1t8RKzddZjSUFMm9rcGBPcR5DSSjPAWNf5c7EE0K2'

# Access:
ACCESS_TOKEN  = '2933481336-lWtyqSrObN7fQFgN9yU6XDmiV4WOlayNoes741Z'
ACCESS_SECRET = 'xgpKoz1Piphhu3DGznZjS5WBGLITyJ0IwKX6U1vVzWSHW'

In [ ]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api

In [ ]:
# We create an extractor object:
extractor = twitter_setup()

# We create a tweet list as follows:
tweets = extractor.user_timeline(screen_name="Xbox", count=200)
print("Number of tweets extracted: {}.\n".format(len(tweets)))

In [ ]:
# We create a pandas dataframe as follows:
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data.head(10))

In [ ]:
data['len']  = np.array([len(tweet.text) for tweet in tweets])
data['ID']   = np.array([tweet.id for tweet in tweets])
data['Date'] = np.array([tweet.created_at for tweet in tweets])
data['Source'] = np.array([tweet.source for tweet in tweets])
data['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
data['RTs']    = np.array([tweet.retweet_count for tweet in tweets])

#  CLEANING THE DATA AND ADDING NEW COLUMN

In [3]:
### SAVING DF TO CSV FILE
data.to_csv("User_Xbox.csv",index=False)

NameError: name 'tweet_text' is not defined

In [3]:
#Checking the length
xbox = pd.read_csv (r'C:\Users\keven\Desktop\User_Xbox.csv')
print(len(xbox))


200


In [4]:
xbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
Tweets    200 non-null object
len       200 non-null int64
ID        200 non-null int64
Date      200 non-null object
Source    200 non-null object
Likes     200 non-null int64
RTs       200 non-null int64
dtypes: int64(4), object(3)
memory usage: 11.1+ KB


In [5]:
#Renaming the column for consistency basis
xbox.rename(columns={'date': 'Date','likes':'Likes','source':'Source','len':'Length'}, inplace=True)

In [6]:
xbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
Tweets    200 non-null object
Length    200 non-null int64
ID        200 non-null int64
Date      200 non-null object
Source    200 non-null object
Likes     200 non-null int64
RTs       200 non-null int64
dtypes: int64(4), object(3)
memory usage: 11.1+ KB


In [7]:
### DROPPIN ALL NAs AND DUPLICATES
xbox.drop_duplicates(keep=False,inplace=True) 


In [8]:
# No duplicate row
print(len(xbox))

200


In [9]:
#### location have a lot of missing values. lets take a deeper look####
xbox.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 7 columns):
Tweets    200 non-null object
Length    200 non-null int64
ID        200 non-null int64
Date      200 non-null object
Source    200 non-null object
Likes     200 non-null int64
RTs       200 non-null int64
dtypes: int64(4), object(3)
memory usage: 12.5+ KB


In [11]:
#Droppin nas
xbox.dropna(inplace=True)
print(xbox.head())

                                              Tweets  Length  \
0  If you were busy playing Cyberpunk and missed ...     135   
1  Better study up on the mantras of survival. Yo...     119   
2  @HbroGaming We're very much looking forward to...      53   
3  @iSentrixx That's so exciting, and those are s...      80   
4     @Fiberyko What can we say, we love a good RPG.      46   

                    ID                 Date               Source  Likes  RTs  
0  1337585464048414721  2020-12-12 02:30:00      Twitter Web App   4147  240  
1  1337577915857289217  2020-12-12 02:00:00      Twitter Web App   1047   32  
2  1337572560213520392  2020-12-12 01:38:43  Sprinklr Publishing      3    0  
3  1337572195216859137  2020-12-12 01:37:16  Sprinklr Publishing      2    0  
4  1337571377524727811  2020-12-12 01:34:01  Sprinklr Publishing     17    0  


In [12]:
#Reindexing
xbox.index = range(len(xbox))
xbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
Tweets    200 non-null object
Length    200 non-null int64
ID        200 non-null int64
Date      200 non-null object
Source    200 non-null object
Likes     200 non-null int64
RTs       200 non-null int64
dtypes: int64(4), object(3)
memory usage: 11.1+ KB


In [13]:
### CLEANING TWEETS FUNCTION ####
import re
def cleanTxt(text):
    text= re.sub(r'@[A-Za-z0-9\_]+','',text) #Removed @mentions
    text= re.sub(r'#','',text)#Remove the '#' symbol
    text= re.sub(r'RT[\s]+','',text)#Remove retweets
    text= re.sub(r'https?:\/\/\S+','',text) #Remove links
    text= re.sub(r'ps5','',text)
    return text

In [15]:
#### CREATING A NEW COLUMN WHERE THE CLEANED TWEETS ARE STORED ###
xbox['Tweets_Cleaned'] = xbox['Tweets'].apply(cleanTxt)

In [16]:
### Remove emojis 
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)


In [17]:
### CREATING A LIST THAT CONTAINS ALL THE REMOVED EMOJIS
list2=[]
for i in xbox.Tweets_Cleaned:
    list2.append(remove_emojis(i))

In [18]:
### CLEANING THE TWEETS FROM EMOJIS
for i in range(len(xbox)):
    xbox.loc[[i],['Tweets_Cleaned']]= list2[i]

In [20]:
### Cleaniing the unclead tweet
xbox.drop(['Tweets'], axis=1, inplace=True)

In [21]:
### unique###
xbox.Source.unique()

array(['Twitter Web App', 'Sprinklr Publishing', 'Twitter Media Studio'],
      dtype=object)

In [22]:
# Replacing not important ones by others
list3=['ios','Android','Web App']

for i in xbox.Source.unique():
    if i not in list3:
        xbox.replace(to_replace=i, value='Others',inplace=True)

In [24]:
#Get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Get Polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [25]:
#Create two new columns
xbox['Subjectivity']=xbox['Tweets_Cleaned'].apply(getSubjectivity)
xbox['Polarity']=xbox['Tweets_Cleaned'].apply(getPolarity)

In [26]:
def getAnalysis(score):
    if score <0:
        return 'Negative'
    elif score==0:
        return 'Neutral'
    else:
        return 'Positive'
xbox['Sentiment']=xbox['Polarity'].apply(getAnalysis)

In [33]:
xbox.ID.unique()

array([1337585464048414721, 1337577915857289217, 1337572560213520392,
       1337572195216859137, 1337571377524727811, 1337570430756929537,
       1337570171641360385, 1337568876268228608, 1337566527126552576,
       1337562816757186560, 1337555264187166721, 1337547720706248705,
       1337532616547831808, 1337529357854707714, 1337525324100358144,
       1337525149663338498, 1337518475158446082, 1337517631440519172,
       1337515313571753986, 1337511628636631042, 1337511568108638208,
       1337502416904671232, 1337502198331285510, 1337494980131053571,
       1337487482712027136, 1337481334772682752, 1337481090710315009,
       1337480453016707076, 1337480040297209857, 1337479828396654592,
       1337479044569370624, 1337478114318970880, 1337477177617604614,
       1337476886524555268, 1337476693049667586, 1337476513936105479,
       1337476328594022402, 1337472220407455754, 1337469131357102086,
       1337465583315501056, 1337457772913364993, 1337457559318405120,
       1337457027648

#  Putting the data into mysql

In [29]:
##### Creating a database #####
mydb=mysql.connector.connect(
    host="localhost",
    user='root',
    password="password")

mycursor = mydb.cursor()



In [30]:
mycursor.execute("CREATE DATABASE Project6")

In [34]:
xbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
Length            200 non-null int64
ID                200 non-null int64
Date              200 non-null object
Source            200 non-null object
Likes             200 non-null int64
RTs               200 non-null int64
Tweets_Cleaned    200 non-null object
Subjectivity      200 non-null float64
Polarity          200 non-null float64
Sentiment         200 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 15.8+ KB


In [35]:
mydb=mysql.connector.connect(
    host="localhost",
    user='root',
    password="password",
    database="Project6")

In [37]:
my_cursor = mydb.cursor()

In [38]:
my_cursor.execute("CREATE TABLE xbox_acc (id BIGINT , Date DATETIME, Likes INT,RTs INT, Source TEXT, Length INT, Tweets_Cleaned TEXT, Subjectivity FLOAT, Polarity FLOAT, Sentiment TEXT, primary key(id))")

In [42]:
def insertVariblesIntoTable(id, Date, Likes, RTs, Source, length, Tweets_Cleaned,Subjectivity,Polarity,Sentiment):
    try:
        connection = mysql.connector.connect(host="localhost",
                                                    user='root',
                                                    password="password",
                                                    database="Project6")
        cursor = connection.cursor()
        mySql_insert_query = """INSERT INTO xbox_acc (id, Date,Likes, RTs, Source, length,Tweets_Cleaned,Subjectivity,Polarity,Sentiment) 
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s)"""

        recordTuple = (id, Date,Likes, RTs, Source, length, Tweets_Cleaned,Subjectivity,Polarity,Sentiment)
        cursor.execute(mySql_insert_query, recordTuple)
        connection.commit()
        print("Record inserted successfully into xbox_acc table")

    except mysql.connector.Error as error:
        print("Failed to insert into MySQL table {}".format(error))

    finally:
        if (connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [43]:
for i in range(len(xbox)):
    insertVariblesIntoTable(int(xbox.loc[i,"ID"]),xbox.loc[i,"Date"], int(xbox.loc[i,"Likes"]), int(xbox.loc[i,"RTs"]),xbox.loc[i,"Source"], int(xbox.loc[i,"Length"]),xbox.loc[i,"Tweets_Cleaned"],xbox.loc[i,"Subjectivity"],xbox.loc[i,"Polarity"], xbox.loc[i,"Sentiment"])


Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inser

Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inser

# SQL Queries can be found in the first python code